## Week 12 Exercise 12.2 Author: Rex Gayas Course & Section: DSC360-T301 Data Mining: Text Analytics an (2243-1) Date: 02 MAR 2024

##### Loading the Dataset

In [62]:
import pandas as pd

# Define the file path
file_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Mining\\Week 5\\twitter_sample.csv"

# Load the dataset into a Pandas DataFrame
tweets_df = pd.read_csv(file_path)

# Display the first few rows of the dataframe to verify
print(tweets_df.head())


                Tweet Id                                          Tweet URL  \
0  "1167429261210218497"  https://twitter.com/animalhealthEU/status/1167...   
1  "1167375334670557185"  https://twitter.com/PennyBrohnUK/status/116737...   
2  "1167237977615097861"  https://twitter.com/lordbyronaf/status/1167237...   
3  "1167236897078480898"  https://twitter.com/CountessDavis/status/11672...   
4  "1167228378191204353"  https://twitter.com/Local12/status/11672283781...   

  Tweet Posted Time (UTC)                                      Tweet Content  \
0    30 Aug 2019 13:30:00  Pets change our lives &amp; become a part of o...   
1    30 Aug 2019 09:55:43  Another spot of our #morethanmedicine bus in #...   
2    30 Aug 2019 00:49:54  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
3    30 Aug 2019 00:45:37  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   
4    30 Aug 2019 00:11:46  What a great team ⁦@HealthSourceOH⁩ ⁦@Local12⁩...   

  Tweet Type                Client  Retweets

##### Preparing the spaCy Pipeline

In [63]:
import spacy

# Load the English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")


spaCy’s small English  language model has been loaded which will be used to process and tokenize the tweets. It creates a Doc object, which is a sequence of tokens that spaCy uses for all its language processing tasks.

##### Defining the Matcher and Patterns for Social Causes

In [64]:
# Initialize the Matcher with the spaCy vocabulary
matcher = Matcher(nlp.vocab)

# Define patterns for social causes
social_cause_patterns = [
    [{"LOWER": "support"}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": "the"}, {"LOWER": "cause"}],
    [{"LOWER": "raise"}, {"LOWER": "awareness"}],
    [{"LOWER": "fight"}, {"LOWER": "against"}],
    [{"LOWER": "advocate"}, {"LOWER": "for"}],
    [{"LOWER": "social"}, {"LOWER": "justice"}],
    [{"LOWER": "human"}, {"LOWER": "rights"}],
    [{"LOWER": "help"}, {"LOWER": "the"}, {"LOWER": "poor"}],
    [{"LOWER": "poverty"}, {"LOWER": "relief"}],
    [{"LOWER": "equality"}],
    [{"LOWER": "environmental"}, {"LOWER": "protection"}],
    [{"LOWER": "climate"}, {"LOWER": "change"}],
    [{"LOWER": "political"}, {"LOWER": "reform"}],
    [{"LOWER": "donate"}, {"LOWER": "to"}],
    [{"LOWER": "charity"}],
    [{"LOWER": "volunteer"}, {"LOWER": "for"}],
    [{"LOWER": "fundraiser"}],
    [{"LOWER": "make"}, {"LOWER": "a"}, {"LOWER": "difference"}]
]

# Add patterns to the matcher
matcher.add("SOCIAL_CAUSE", social_cause_patterns)


Defined a set of patterns that are likely to capture references to social causes within tweets. These patterns are based on common phrases and wordings that are associated with social advocacy or mentions of social issues.

##### Extracting Social Cause Mentions from the Dataset

In [69]:
# Function to process tweets and extract matches
def extract_social_causes(tweet_content):
    # Process the tweet content with the spaCy model
    doc = nlp(tweet_content)
    # Find matches in the doc
    matches = matcher(doc, as_spans=True)
    # Concatenate all matched text into a single string separated by commas
    return ', '.join([span.text.strip() for span in matches if span.text.strip()])

# Apply the function to the 'Tweet Content' column to extract matches
tweets_df['Matched Text'] = tweets_df['Tweet Content'].apply(extract_social_causes)

# Filter out rows with no matches
tweets_with_matches = tweets_df[tweets_df['Matched Text'] != '']

# Create a new DataFrame with the relevant columns
matches_df = tweets_with_matches[['Name', 'Matched Text']].copy()

# Add the 'Identifier' column with the constant value 'SOCIAL_CAUSE'
matches_df['Identifier'] = 'SOCIAL_CAUSE'

# Display the DataFrame with the matched social cause phrases
matches_df.head()


,Name,Matched Text,Identifier
125,NOAH,raise awareness,SOCIAL_CAUSE
359,Ibrahima KOUMA,environmental protection,SOCIAL_CAUSE
380,AnimalhealthEurope,environmental protection,SOCIAL_CAUSE


Defined a function “extract_social_causes” that takes the tweet content, processes it with the NLP model to create a doc object, and then finds all spans in the doc that match predefined patterns. Included only spans where the text is not just whitespace.
Applied this to “Tweet Content” column of DataFrame and created a new column “Matched Text” that contains the matched phrases for each tweet. Filtered out any tweets that didn't match any patterns and created a new “DataFrame matches_df” containing the tweet author's name, the matched text, and a constant identifier denoting that the match is related to a social cause.
